## Attractor Landscape analysis

This is an example of how Boolean network model was analyzed in our paper.

We provide the codes for 'Attractor landscape analysis', 'Perturbation analysis', and 'Analysis of network dynamics' (Please refer to the Method section).


In [ ]:
import numpy as np
import pandas as pd
import itertools
import networkx as nx
import copy
import os

from pyboolnet.file_exchange import bnet2primes, primes2bnet
from pyboolnet.interaction_graphs import primes2igraph
from pyboolnet.state_transition_graphs import primes2stg
from pyboolnet.attractors import compute_attractors_tarjan

from modules.attractorSim import rand_initial_states, compute_attractor_from_primes, compute_phenotype, Simulation
from modules.printlogic import print_node_logic

In [ ]:
model_file = 'emt_randomized_logic_topology_preserved_complex.bnet'
primes = bnet2primes(model_file)
nodeList = list(primes.keys())
graph = primes2igraph(primes)
update_mode = "synchronous"

nodes_order_er = sorted(primes.keys())
print("총 노드 수:", len(nodes_order_er))
print("앞 40개 노드:", nodes_order_er[:40])

# ----------------------------------------------------
# 랜덤 네트워크를 위한 Phenotype/Markers 정의 (EMT 네트워크 기준 재사용)
# ----------------------------------------------------

# 1. 가상의 마커 노드 선택
# ER 네트워크의 노드 중에서 2개를 선택 (첫 번째와 두 번째 노드)
if len(nodes_order_er) < 2:
    raise ValueError("랜덤 네트워크의 노드가 너무 적어서 마커를 지정할 수 없습니다.")

virtual_Ecad_node = nodes_order_er[0] # Node1 (보통 Node0/Node1 등으로 시작할 것임)
virtual_ZEB_node = nodes_order_er[1]  # Node10

print(f"가상 E-cad 노드: {virtual_Ecad_node}, 가상 ZEB1 노드: {virtual_ZEB_node}")

# 2. markers 변수 (기존 markers 변수 대체)
markers_er = [virtual_Ecad_node, virtual_ZEB_node]

# 3. phenotype 정의 (EMT 논문 기준과 동일하게)
phenotype_er = {
    'P_pheno_1': {virtual_Ecad_node:1, virtual_ZEB_node:0},   # epithelial
    'P_pheno_2': {virtual_Ecad_node:0, virtual_ZEB_node:1},  # mesenchymal
    'P_pheno_3': {virtual_Ecad_node:1, virtual_ZEB_node:1},  # hybrid (E-cad + / ZEB1 +)
    'P_pheno_4' : {virtual_Ecad_node:0, virtual_ZEB_node:0}   # hybrid (E-cad - / ZEB1 -)
}

# 4. phenotypeAnnot 정의 (EMT 논문 기준과 동일하게)
phenotypeAnnot_er = {
    'P_pheno_1': -1,
    'P_pheno_2': 1,
    'P_pheno_3': 0,
    'P_pheno_4': 0
}

print("랜덤 네트워크를 위한 phenotype/markers 정의 완료.")
# --- 끝 ---


if 2**len(nodeList) >= 100000: num_init = 100000
else:  num_init = 2**len(nodeList)
initState = rand_initial_states(num_init, len(nodeList))

총 노드 수: 31
앞 40개 노드: ['Node1', 'Node10', 'Node11', 'Node12', 'Node13', 'Node14', 'Node15', 'Node16', 'Node17', 'Node18', 'Node19', 'Node2', 'Node20', 'Node21', 'Node22', 'Node23', 'Node24', 'Node25', 'Node26', 'Node27', 'Node28', 'Node29', 'Node3', 'Node30', 'Node31', 'Node4', 'Node5', 'Node6', 'Node7', 'Node8', 'Node9']
가상 E-cad 노드: Node1, 가상 ZEB1 노드: Node10
랜덤 네트워크를 위한 phenotype/markers 정의 완료.


### Node perturbation analysis

In [17]:
save_dir = './result/EMT/' 
save_perturbname = save_dir + 'Random_simul_result_1.csv'

In [18]:
fix_dict = {} # 또는 가상의 RAS 노드 고정

In [19]:
perturb_p = pd.DataFrame([]) # average activities of the marker nodes
perturb_s = pd.DataFrame([]) # network stabiltiy 

fix_dict_tmp = copy.deepcopy(fix_dict)

print(fix_dict_tmp) # 현재 적용할 고정 노드 출력

primes_new, pheno_df, att_ave_pd, attrs_dict = Simulation(fix_dict_tmp, primes, update_mode, initState, phenotype_er, phenotypeAnnot_er)   

att_ave_pd.to_csv("rand_att_ave_pd_1.csv")
pheno_df.to_csv("rand_pheno_df_1.csv")

{}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.

--- [Simulation Function] Detected Attractors for Current Perturbation ---
  > Attractor ID 0 (Steady State, Basin 51.19%)
    State Str:  1000001110010111100000010010111
  > Attractor ID 1 (Steady State, Basin 5.19%)
    State Str:  1101111100110000100000001010010
  > Attractor ID 2 (Steady State, Basin 20.86%)
    State Str:  1000111100110000100000000010010
  > Attractor ID 3 (Cyclic, Length 2, Basin 22.76%)
    Cycle Start Str:  1000111100110000100000001010010
       Step 1 Str:  1000111100110000100000001010010
       Step 2 Str:  1101111100110000100000000010010
--- End of Attractor Listing ---
Attractor simulation time : 10.928932666778564
              Ratio
phenotype          
P_pheno_1  0.720527
P_pheno_3  0.279473


### Node 하나씩 제거

In [20]:
# --------------- 로직만 없애는 시뮬레이션 프레임워크 ---------------
# 이 셀은 이전 코드 블록(RAS=1 기본 시뮬레이션) 다음에 실행
import time # time.time() 사용을 위해 필요
import copy # copy.deepcopy() 사용을 위해 필요
import pandas as pd # 결과 저장을 위해 필요
import os # 디렉토리 생성을 위해 필요

# 전역 변수로 이미 정의되어 있어야 할 변수들 확인:
# primes, nodes_order, update_mode, initState, phenotype, phenotypeAnnot, Simulation
fix_dict_for_single_run = {}

print("\n--- 노드별 로직 제거(Logic Disruption) 분석 시작 ---")

# 결과 저장 디렉토리
results_dir_logic_disruption = './logic_disruption_results'
os.makedirs(results_dir_logic_disruption, exist_ok=True)

logic_disruption_results = []

# 원본 primes를 백업 (매 루프마다 초기 상태로 돌려야 하므로)
# primes는 `model_file = network_dir + 'EMT_Network.bnet'` 이후에 로드된 원본
original_primes_backup = copy.deepcopy(primes)

for node_to_disrupt_logic in nodes_order_er:
    # RAS 노드의 로직은 건드리지 않음 (이미 고정했거나 다른 중요한 기본 노드일 수 있으므로)
    # 필요하다면 다른 특정 노드도 제외 가능
    if node_to_disrupt_logic == 'RAS': # and 'RAS' in fix_dict_for_single_run:
        print(f"\n--- RAS 노드 로직은 건드리지 않음 (기본 고정값 사용) ---")
        continue

    # 1. primes를 현재 루프에 맞게 복사 (항상 원본 primes에서 시작)
    current_primes_for_disruption = copy.deepcopy(original_primes_backup)

    # 2. 해당 노드의 '로직(Boolean function)'을 제거
    #    -> 이 노드는 이제 다른 노드들의 영향을 받지 않고 '항상 1'을 출력한다고 가정
    #    사수님 의도에 따라: '실수로 고려 못했을 경우' = '기능적으로 비활성화'
    #    이를 모델에서는 그 노드가 항상 1 (ON) 값을 내게 함.
    constant_output_value = 1 # 로직 제거된 노드가 '항상 1'을 출력한다고 가정

    # current_primes_for_disruption에서 node_to_disrupt_logic의 Boolean 함수를 변경
    # primes[노드명] = [ [입력_노드_리스트], {(): 출력_값} ] 형태로 변경
    # 입력_노드_리스트를 빈 리스트 `[]`로 만들어 더 이상 입력에 의존하지 않게 하고,
    # {(): 출력_값}으로 항상 `constant_output_value`를 반환하게 함.
    current_primes_for_disruption[node_to_disrupt_logic] = [[], {(): constant_output_value}]
    
    print(f"\n--- 시뮬레이션: '{node_to_disrupt_logic}' 로직 제거 (항상 {constant_output_value} 출력) ---")
    print_node_logic(node_to_disrupt_logic, current_primes_for_disruption)

    # 3. Simulation 호출에 넘길 고정 노드 딕셔너리 생성
    #    RAS 고정 조건에 로직 제거된 노드도 그 고정 값으로 추가
    fix_dict_for_sim_call = copy.deepcopy(fix_dict_for_single_run) # {'RAS':1}
    fix_dict_for_sim_call.update({node_to_disrupt_logic: constant_output_value}) # 예: {'RAS':1, 'AKT':0}

    # Simulation 함수 호출
    try:
        primes_after_disruption, pheno_df_logic, att_ave_pd_logic, attrs_dict_logic = Simulation(
            fix_dict_for_sim_call,       # 이번 시뮬레이션의 고정 노드
            current_primes_for_disruption, # 로직 제거된 노드 정의가 바뀐 primes
            update_mode,
            initState,
            phenotype_er,
            phenotypeAnnot_er
        )
        
        # 결과 수집 (KO 시뮬레이션과 동일한 지표 사용)
        logic_disruption_results.append({
            'disrupted_node': node_to_disrupt_logic,
            'fixed_value_after_disruption': constant_output_value,
            'attractor_counts': len(attrs_dict_logic.get('attractors', {})),
            'phenotype_ratios': pheno_df_logic.T.to_dict('records')[0],
            'att_ave_pd_values': att_ave_pd_logic.T.to_dict('records')[0]
        })
    except Exception as e:
        print(f"에러 발생: '{node_to_disrupt_logic}' 로직 제거 시뮬레이션 중 - {e}")
        logic_disruption_results.append({
            'disrupted_node': node_to_disrupt_logic,
            'fixed_value_after_disruption': constant_output_value,
            'error': str(e),
            'attractor_counts': np.nan 
        })

# 결과를 Pandas DataFrame으로 정리
logic_disruption_df = pd.DataFrame(logic_disruption_results)
logic_disruption_df.set_index('disrupted_node', inplace=True)
logic_disruption_df.to_csv(os.path.join(results_dir_logic_disruption, 'rand_node_logic_disruption_analysis_results_1.csv'))
print(f"\n노드 로직 제거 분석 결과 저장 완료: {os.path.join(results_dir_logic_disruption, 'rand_node_logic_disruption_analysis_results_1.csv')}")


--- 노드별 로직 제거(Logic Disruption) 분석 시작 ---

--- 시뮬레이션: 'Node1' 로직 제거 (항상 1 출력) ---

--- 노드 'Node1'의 Boolean Function ---
  입력 노드들: 없음 (상수)
  진리표 (Truth Table): {(): 1}
  ==> 확인: 이 노드는 입력에 무관하게 항상 '1'을(를) 출력하는 상수 함수입니다 (로직 제거됨).
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.

--- [Simulation Function] Detected Attractors for Current Perturbation ---
  > Attractor ID 0 (Steady State, Basin 51.20%)
    State Str:  1000001110010111100000010010111
  > Attractor ID 1 (Steady State, Basin 5.27%)
    State Str:  1101111100110000100000001010010
  > Attractor ID 2 (Steady State, Basin 20.74%)
    State Str:  1000111100110000100000000010010
  > Attractor ID 3 (Cyclic, Length 2, Basin 22.78%)
    Cycle Start Str:  1000111100110000100000001010010
       Step 1 Str:  1000111100110000100000001010010
       Step 2 Str:  1101111100110000100000000010010
--- End of Attractor Listing ---
Attractor simulation time : 10.617400884628296
    